In [0]:
import numpy as np, pandas as pd
from keras.layers import Dense, BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.layers.core import Activation
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.applications.densenet import DenseNet121

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
model = DenseNet121(include_top=False, weights='imagenet', input_shape=(128,128,3), pooling = 'max')

model.trainable = False

set_trainable = False
for layer in model.layers:
  if layer.name.startswith("conv5"):
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

x = model.output

bin_classifier = Dense(64, kernel_regularizer = l2(0.05))(x)
bin_classifier = Activation("relu")(bin_classifier)
bin_classifier = BatchNormalization(axis = -1)(bin_classifier)
bin_classifier = Dense(1, kernel_regularizer = l2(0.05))(bin_classifier)
bin_classifier = Activation("sigmoid", name = "bin_classifier")(bin_classifier)

reg_head = Dense(64, kernel_regularizer = l2(0.05))(x)
reg_head = Activation("relu")(reg_head)
reg_head = BatchNormalization(axis = -1)(reg_head)
reg_head = Dense(1, name = "reg_head", kernel_regularizer = l2(0.05))(reg_head)

base_model = Model(inputs = model.input, outputs = [bin_classifier, reg_head])

In [0]:
images = np.load('drive/My Drive/imfdb_dataset/imfdb_images.npy')
gender = np.load('drive/My Drive/imfdb_dataset/imfdb_gender_labels.npy')
age = np.load('drive/My Drive/imfdb_dataset/imfdb_age_labels.npy')

In [0]:
images = images.astype("float") / 255.0

In [0]:
(x_train, x_test, gender_train, gender_test, age_train, age_test) = train_test_split(images, gender, age, test_size = 0.2, random_state = 24)

In [0]:
lb = LabelBinarizer()
gender_train = lb.fit_transform(gender_train)
gender_test = lb.fit_transform(gender_test)

In [0]:
loss_weights = {'reg_head': 1., 'bin_classifier': 3.}
losses = {'reg_head': 'mse', 'bin_classifier': 'binary_crossentropy'}

In [0]:
base_model.compile(optimizer = "nadam", loss = losses, loss_weights = loss_weights, metrics = ["accuracy"])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
filepath = "weight-improvement-{epoch:02d}-{loss:4f}.hd5"
checkpoint = ModelCheckpoint(filepath, monitor = 'val_bin_classifier_acc', verbose = 1)
callbacks = [checkpoint]

In [0]:
epochs = 25
batch_size = 16

In [0]:
base_model.fit(x_train, {'reg_head': age_train, 'bin_classifier': gender_train}, batch_size = batch_size, epochs=epochs, callbacks= callbacks,  validation_data=(x_test, {'reg_head': age_test, 'bin_classifier': gender_test}))

Train on 23232 samples, validate on 5808 samples
Epoch 1/25
23232/23232 [==============================] - 1599s 69ms/step - loss: nan - bin_classifier_loss: nan - reg_head_loss: nan - bin_classifier_acc: 0.8170 - reg_head_acc: 0.3534 - val_loss: nan - val_bin_classifier_loss: nan - val_reg_head_loss: nan - val_bin_classifier_acc: 0.0000e+00 - val_reg_head_acc: 0.0000e+00

Epoch 00001: saving model to weight-improvement-01- nan.hd5


KeyboardInterrupt: ignored

In [0]:
np.unique(age_train)

array([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 64, 66], dtype=int64)